In [ ]:
import streamlit as st
import urllib.parse
from datetime import datetime
import utils.auth as auth   # make sure auth.py is in a folder called "utils"

st.set_page_config(page_title="Home", layout="wide")

# --- Title & Date ---
today = datetime.today().strftime("%A, %B %d, %Y")
st.markdown(f"<div style='text-align:right; color:#555;'>{today}</div>", unsafe_allow_html=True)
st.title("🏠 Home Page")

# --- Session state init ---
if "logged_in" not in st.session_state:
    st.session_state.logged_in = False
    st.session_state.user_info = None

# --- Login form ---
if not st.session_state.logged_in:
    st.subheader("🔑 Login")

    username = st.text_input("Username")
    password = st.text_input("Password", type="password")

    if st.button("Login"):
        if auth.verify_user(username, password):
            user_info = auth.get_user_info(username)
            if user_info:
                st.session_state.logged_in = True
                st.session_state.user_info = user_info
                st.success(f"✅ Welcome, {user_info['name'] or user_info['username']}!")
                st.rerun()  # rerun to refresh UI
            else:
                st.error("⚠️ یوزر یا پسورد اشتباه است")
        else:
            st.error("❌ یوزر یا پسورد اشتباه است")

else:
    # --- Show logged-in info ---
    user_info = st.session_state.user_info
    st.markdown(f"""
    <div style="margin-bottom:20px; padding:10px; border:1px solid #ccc; border-radius:10px;">
        <p>👤 <b>User:</b> {user_info['username']}</p>
        <p>📛 <b>Name:</b> {user_info['name']}</p>
        <p>🏢 <b>Department:</b> {user_info['department']}</p>
        <p>🆔 <b>Personnel Number:</b> {user_info['personnel_number']}</p>
        <p>🔑 <b>Admin:</b> {"Yes" if user_info['is_admin'] else "No"}</p>
    </div>
    """, unsafe_allow_html=True)

    # --- Encode multiple query params ---
    query_params = {
        "username": user_info["username"],
        "name": user_info["name"] or "",
        "department": user_info["department"] or ""
    }
    encoded_params = urllib.parse.urlencode(query_params, quote_via=urllib.parse.quote)
   # st.info(encoded_params)
    # --- Navigation Links ---
    st.subheader("🖇️ Navigate to Pages")
    
    # Daily Jobs link
    st.markdown(
        f"<a href='/daily_jobs?{encoded_params}' style='font-size:18px; color:blue; text-decoration:underline;'>📋 Daily Jobs</a>",
        unsafe_allow_html=True
    )
    
    # Object Details link
    st.markdown(
        f"<a href='/object?{encoded_params}' style='font-size:18px; color:orange; text-decoration:underline;'>🔍 Object Details</a>",
        unsafe_allow_html=True
    )
    # st.markdown("<hr style='border: 1px solid #e67e22; margin:1px 0;'>", unsafe_allow_html=True)  # orange line
    st.markdown("""
            <hr style='border: 0.5px dotted rgba(128,128,128,0.3); margin-top: -5px; margin-bottom: 5px;'>
            """, unsafe_allow_html=True)


    # --- Define New Tag link ---
    st.markdown(
        f"<a href='/Define_New_Tag?{encoded_params}' style='font-size:18px; color:purple; text-decoration:underline;'>🆕 Define New Tag</a>",
        unsafe_allow_html=True
    )
    st.markdown("""
        <hr style='border: 0.5px dotted rgba(128,128,128,0.3); margin-top: -5px; margin-bottom: 5px;'>
    """, unsafe_allow_html=True)

    
    # User Management link (only for admin)
    if user_info.get("is_admin"):
        st.markdown(
            f"<a href='/User_Management?{encoded_params}' style='font-size:18px; color:green; text-decoration:underline;'>👤 User Management</a>",
            unsafe_allow_html=True
        )

        #st.markdown("<hr style='border: 1px solid #2ecc71; margin:1px 0;'>", unsafe_allow_html=True)  # green line
    st.markdown("""
            <hr style='border: 0.5px dotted rgba(128,128,128,0.3); margin-top: -5px; margin-bottom: 5px;'>
            """, unsafe_allow_html=True) 

    # --- Logout ---
    if st.button("Logout"):
        st.session_state.logged_in = False
        st.session_state.user_info = None
        st.rerun()
